In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob
import keras
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.
C:\Users\micke\Miniconda3\envs\rcnn_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\micke\Miniconda3\envs\rcnn_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\micke\Miniconda3\envs\rcnn_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\micke\Miniconda3\envs\r

In [2]:
import tensorflow as tf

cf = tf.ConfigProto()
cf.gpu_options.allow_growth = True
sess = tf.Session(config=cf)

In [3]:
MASK_TRAIN_PATH = r"D:\Gproject\DataSet\tcia_data\dataset_dir\mask_train\*.png"
MASK_TEST_PATH = r"D:\Gproject\DataSet\tcia_data\dataset_dir\mask_test\*.png"

In [4]:
MASK_TRAIN_PATHS = glob.glob(MASK_TRAIN_PATH)
MASK_TEST_PATHS = glob.glob(MASK_TEST_PATH)

In [16]:
# 이미지의 평균 구하기
img_x = 0
img_y = 0
for i in MASK_TRAIN_PATHS:
    img = plt.imread(i)
    img_x += img.shape[0]
    img_y += img.shape[1]

# 410.0998178506375
print(img_x/2745)

# 421.0761384335155
print(img_y/2745)

1125724
1155854


In [5]:
#train set
count = 0
dsize =(224,224)
train_x = np.array
train_y = np.array([])

for i in MASK_TRAIN_PATHS:
    
    img = cv.imread(i,cv.COLOR_RGB2BGRA)
    
    if img.shape[0] > dsize[0] and img.shape[1] > dsize[1]:
        resize_img = cv.resize(img,dsize,interpolation = cv.INTER_CUBIC)
        
    else:
        resize_img = cv.resize(img,dsize,interpolation = cv.INTER_AREA)

    resize_img_newaxis = resize_img[np.newaxis,...]
    count +=1
    #print(count)
    
    if count == 1:
        train_x = resize_img_newaxis
    else:
        train_x = np.concatenate((train_x, resize_img_newaxis))
    
    abnormality = i.split("\\")[-1][:4]
    if abnormality == 'Calc':
        train_y = np.concatenate((train_y,[0]))
    elif abnormality == 'Mass':
        train_y = np.concatenate((train_y,[1]))
    else:
        print(i)
        
train_x = train_x[...,np.newaxis]
print(train_x.shape)
print(train_y.shape)

(2745, 224, 224, 1)
(2745,)


In [6]:
train_y = to_categorical(train_y,num_classes=2,dtype=np.uint8)

In [7]:
#val set
count = 0
dsize =(224,224)
test_x = np.array
test_y = np.array([])

for i in MASK_TEST_PATHS:
    
    img = cv.imread(i,cv.COLOR_RGB2BGRA)
    
    if img.shape[0] > dsize[0] and img.shape[1] > dsize[1]:
        resize_img = cv.resize(img,dsize,interpolation = cv.INTER_CUBIC)
        
    else:
        resize_img = cv.resize(img,dsize,interpolation = cv.INTER_AREA)

    resize_img_newaxis = resize_img[np.newaxis,...]
    count +=1
    #print(count)
    
    if count == 1:
        test_x = resize_img_newaxis
    else:
        test_x = np.concatenate((test_x, resize_img_newaxis))
    
    abnormality = i.split("\\")[-1][:4]
    if abnormality == 'Calc':
        test_y = np.concatenate((test_y,[0]))
    elif abnormality == 'Mass':
        test_y = np.concatenate((test_y,[1]))
    else:
        print(i)
test_x = test_x[...,np.newaxis]
print(test_x.shape)
print(test_y.shape)

(690, 224, 224, 1)
(690,)


In [8]:
test_y = to_categorical(test_y,num_classes=2,dtype=np.uint8)

In [9]:
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import callbacks
from keras.applications import ResNet50V2

In [10]:
csv_logger = callbacks.CSVLogger('./log_classification/00log.scv')

In [11]:
def bulid_ResNet50V2(input_shape=None):
    base_model = ResNet50V2(include_top=False, weights=None, input_shape=input_shape)

    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    
    x = layers.Dense(512, activation='relu', name='fc1')(x)
    x = layers.Dense(256, activation='relu', name='fc2')(x)
    main_output = layers.Dense(2, name='fc3',activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=main_output, name='resnet50_v2')

    return model

In [12]:
model = bulid_ResNet50V2(input_shape=(224,224,1))

In [31]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy, metrics=['acc'])

In [17]:
train_x.shape

(2745, 224, 224, 1)

In [21]:
train_root = r"D:\Gproject\DataSet\tcia_data\dataset_dir\mask_train"
test_root = r"D:\Gproject\DataSet\tcia_data\dataset_dir\mask_test"

In [32]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
test_datagen = ImageDataGenerator()

#train_generator = train_datagen.flow_from_directory(train_root,target_size=(244,244),batch_size=16,interpolation="bilinear")
#test_generator = train_datagen.flow_from_directory(test_root,target_size=(244,244),batch_size=16,interpolation="bilinear")

train_generator = train_datagen.flow(x=train_x ,y=train_y,batch_size=16)
test_generator = test_datagen.flow(x= test_x,y=test_y,batch_size=16)

In [33]:
hist = model.fit(train_generator,steps_per_epoch=200,
                 epochs=40,validation_data=(test_generator),validation_steps=20,
                 callbacks=[csv_logger])

Epoch 1/40
200/200 [==============================] - 30s 148ms/step - loss: 0.3928 - acc: 0.8154 - val_loss: 1.0387 - val_acc: 0.6031
Epoch 2/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3881 - acc: 0.8284 - val_loss: 0.4818 - val_acc: 0.7469
Epoch 3/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3887 - acc: 0.8290 - val_loss: 2.3017 - val_acc: 0.4739
Epoch 4/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3564 - acc: 0.8443 - val_loss: 1.5305 - val_acc: 0.6750
Epoch 5/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3648 - acc: 0.8412 - val_loss: 0.6338 - val_acc: 0.6536
Epoch 6/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3511 - acc: 0.8503 - val_loss: 1.2494 - val_acc: 0.6219
Epoch 7/40
200/200 [==============================] - 24s 122ms/step - loss: 0.3402 - acc: 0.8434 - val_loss: 0.4597 - val_acc: 0.7582
Epoch 8/40
200/200 [==============================] - 2

In [34]:
model.save(r"./log_classification/00model.h5")